# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Libraries

In [ ]:
import torch
import os
import cv2
import os
import matplotlib.pyplot as plt

# Dataset Details

In [ ]:
# Define the folder path
folder_path = '/content/drive/MyDrive/MLR/S3R2-KIIT'
files = os.listdir(folder_path)

for file in files:
    print(file)

In [ ]:
# Define the paths for train, valid, and test directories
base_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT'
dirs = ['train', 'valid', 'test']

for dir_name in dirs:
    dir_path = os.path.join(base_dir, dir_name)
    print(f"\nDirectory: {dir_name}")
    for subfolder in os.listdir(dir_path):
        subfolder_path = os.path.join(dir_path, subfolder)

        if os.path.isdir(subfolder_path):
            num_files = len([f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))])
            print(f"  Subfolder: {subfolder}, Number of files: {num_files}")


In [ ]:
yaml_file_path = '/content/drive/MyDrive/MLR/S3R2-KIIT/S3R2_data.yml'
with open(yaml_file_path, 'r') as file:
    yaml_content = file.read()

print(yaml_content)


Let's see the information of a annotation file...

In [ ]:
label_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT/test/labels'
label_file = 'image_s3r2_kiit_116.txt'

label_path = os.path.join(label_dir, label_file)

if not os.path.exists(label_path):
    print(f"Label file not found: {label_path}")
else:
    with open(label_path, 'r') as f:
        lines = f.readlines()

    print(f"Label file: {label_file}")
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())

        print(f"Class ID: {int(class_id)}")
        print(f"Bounding Box: x_center={x_center}, y_center={y_center}, width={width}, height={height}")
        print("-" * 50)

In [ ]:
# Class names as per our dataset
class_names = [
    "Artilary",             # ID 0
    "Missile",              # ID 1
    "Radar",                # ID 2
    "M. Rocket Launcher",   # ID 3
    "Soldier",              # ID 4
    "Tank",                 # ID 5
    "Vehicle"               # ID 6
]

# Visualization

In [ ]:
image_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT/test/images'
annotation_dir = '/content/drive/MyDrive/MLR/S3R2-KIIT/test/labels'

def visualize_image_with_annotations(image_name):
    image_path = os.path.join(image_dir, image_name)
    img = cv2.imread(image_path)

    if img is None:
        print(f"Image not found: {image_path}")
        return

    annotation_name = os.path.splitext(image_name)[0] + '.txt'
    annotation_path = os.path.join(annotation_dir, annotation_name)

    if not os.path.exists(annotation_path):
        print(f"Annotation not found: {annotation_path}")
        return

    img_height, img_width = img.shape[:2]

    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.strip().split())

        x_center = int(x_center * img_width)
        y_center = int(y_center * img_height)
        width = int(width * img_width)
        height = int(height * img_height)

        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)

        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=(0, 255, 0), thickness=2)

        class_name = class_names[int(class_id)]

        cv2.putText(img, class_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 8))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

visualize_image_with_annotations('image_s3r2_kiit_1297.jpeg')  # Image to Visualize


In [ ]:
visualize_image_with_annotations('image_s3r2_kiit_150.jpeg')

In [ ]:
visualize_image_with_annotations('image_s3r2_kiit_1591.jpeg')

# Cheking for GPU

In [ ]:
if torch.cuda.is_available():
    gpu_info = torch.cuda.get_device_properties(0)
    print(f"GPU is available: {gpu_info.name}")
    print(f"Total Memory: {gpu_info.total_memory / (1024**3):.2f} GB")
    print(f"CUDA Capability: {gpu_info.major}.{gpu_info.minor}")
else:
    print("GPU not available, running on CPU.")


In [ ]:
!nvidia-smi

# Setting up YOLOV5 small...

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Using the YOLOv5_SMALL model

In [ ]:

!python train.py --img 640 --batch 16 --epochs 100 --data /content/drive/MyDrive/MLR/S3R2-KIIT/S3R2_data.yml --cfg yolov5s.yaml --weights yolov5s.pt --name /content/drive/MyDrive/MLR/S3R2-KIIT --cache


In [ ]:
!python val.py --weights /content/drive/MyDrive/MLR/S3R2-KIIT4/weights/best.pt --data /content/drive/MyDrive/MLR/S3R2-KIIT/S3R2_data.yml --img 640

# Detection

Single Image

In [ ]:
!python detect.py --weights /content/drive/MyDrive/MLR/S3R2-KIIT4/weights/best.pt --img 640 --conf 0.2 --source /content/drive/MyDrive/MLR/S3R2-KIIT/test/images/image_s3r2_kiit_1244.jpeg

In [ ]:

#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp6/image_s3r2_kiit_1244.jpeg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

Multiple Image

In [ ]:

!python detect.py --weights /content/drive/MyDrive/MLR/S3R2-KIIT4/weights/best.pt --source /content/drive/MyDrive/MLR/test_images --img 640 --conf 0.25 --save-txt --save-conf

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

image_folder = 'runs/detect/exp/'

image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.webp'))]


for image_file in image_files:
    img_path = os.path.join(image_folder, image_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.title(image_file)
    plt.axis('off')
    plt.show()

In [ ]:
!python detect.py --weights /content/drive/MyDrive/MLR/S3R2-KIIT4/weights/best.pt --img 640 --conf 0.1 --source /content/drive/MyDrive/MLR/S3R2-KIIT/test/images/image_s3r2_kiit_1358.jpeg


In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp7/image_s3r2_kiit_1358.jpeg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")